# Quantifying galaxy environment

notebook by _Alex Malz (GCCL@RUB)_, _Kara Ponder (UC Berkeley)_, _Ben Moews (Edinburgh)_, _Stephen Thorp ()_, add your name here

In [ ]:
from astropy.io import fits
import corner
import environment as galenv
import matplotlib.pyplot as plt
import multiprocessing as mp
import numpy as np
import os
import pandas as pd

%matplotlib inline
np.seed = 42

We want to get the spectra of galaxies matching conditions found [here](http://www.gama-survey.org/dr3/schema/table.php?id=31).

In [ ]:
with fits.open('SpecObj.fits') as hdul:
    hdul.info()
#     print(hdul[1].header)
    zdf = pd.DataFrame(np.array(hdul[1].data).byteswap().newbyteorder())
    print(zdf.columns)
#     df.index = df['CATAID']

GAMA did calculate some environment measures for us, but only on a small subset of galaxies.

In [ ]:
# with fits.open('EnvironmentMeasures.fits') as hdul:
#     hdul.info()
# #     print(hdul[1].header)
#     envdf = pd.DataFrame(np.array(hdul[1].data).byteswap().newbyteorder())
#     print(envdf.columns)
# #     envdf.index = envdf['CATAID']

In [ ]:
# df = pd.merge(envdf, zdf, on='CATAID')
df = zdf

## Select spectra by redshift and field

Each galaxy in the GAMA catalog has a spectroscopically confirmed redshift.  
We're going to match these redshifts to the snapshots of the particle data.

Add in a cutoff at lowest redshift at 0.023 and highest at 3.066
- Came to this by what the distance is between 0.042 and it's upper endpoint
- Added an upper cutoff as well of 3.066 (by same way for equal redshift space on either side)

In [ ]:
z_SLICS = np.array([0.042, 0.080, 0.130, 0.221, 0.317, 0.418, 0.525, 0.640, 0.764, 0.897, 
           1.041, 1.199, 1.372, 1.562, 1.772, 2.007, 2.269, 2.565, 2.899])
z_mids = (z_SLICS[1:] + z_SLICS[:-1]) / 2.
z_bins = np.insert(z_mids, 0, 0.023)
z_bins = np.append(z_bins, 3.066)
print(z_bins)
plt.hist(df['Z'], bins=z_bins)
plt.semilogy()
plt.xlabel('redshift')
plt.ylabel('number of galaxies')

The histogram of redshift is skewed by the use of `z=10` as a placeholder for not having a secure redshift.  
GAMA has a quality flag we can use to filter for redshifts that were considered of sufficient quality for science use, which they define as `NQ > 2`.

In [ ]:
moar_bins = np.arange(z_bins[0], z_bins[-1] + z_bins[1], z_bins[1])
for i in range(5):
    quality = df.loc[df['NQ'] == i+1, 'Z']
    plt.hist(quality, alpha=0.5, label=str(i+1), bins=moar_bins)
plt.legend(loc='upper right')
plt.semilogy()
plt.xlim(moar_bins[0], moar_bins[-1])
plt.xlabel('Z')
plt.ylabel('number of galaxies')
plt.title('redshift distributions by quality flag "NQ"')

GAMA observed galaxies in four disjoint regions of the sky.
Since environment is about the immediate vicinity of each galaxy, we'll have to divide the galaxies by region, effectively building our redshift-environment-color distribution separately for each region before combining those findings.

In [ ]:
corner.corner(np.array([df['RA'], df['DEC']]).T, labels=['RA', 'DEC'], show_titles=True)

In [ ]:
RA_bin_ends = [0., 80., 160., 200., 360.]
subsamples, lens = [], []
for j in range(len(z_bins)-1):
    for i in range(len(RA_bin_ends)-1):
        subsample = df.loc[(df['RA'] >= RA_bin_ends[i]) & (df['RA'] < RA_bin_ends[i+1]) 
                             & (df['NQ'] > 2) & (df['Z'] >= z_bins[j]) & (df['Z'] < z_bins[j+1]), 
                             ['CATAID', 'RA', 'DEC', 'Z', 'NQ']]
        nn = len(subsample)
        if nn > 0:
            lens.append(nn)
            subsamples.append(subsample)

In [ ]:
print(lens)

In [ ]:
np.shape(subsample)

In [ ]:
chosen_field = np.argmax(lens)
print(chosen_field)

In [ ]:
datum = np.vstack((subsamples[chosen_field]['DEC'], [subsamples[chosen_field]['RA']])).T

## Number of neighbors within a distance

Within each field, we can quantify the density of the local region around each galaxy, which is really what the notion of "galaxy environment" is getting at.
We're going to use the number of neighboring galaxies at each of several given distances in angular coordinates, so as not to incur the computational cost of calculating the distances between all galaxies.

In [ ]:
help(galenv)

### Choose some reasonable radii

Our angular positions are in degrees.
The distances will be in bogus units because the code normalizes to the radius of the Earth, but we only need the number of neighbors within an angular distance, so it should be fine.

In [ ]:
chosen_ind = np.random.randint(0, len(datum), 1)[0]
print(chosen_ind)
try_distances = np.flip(np.geomspace(0.01, 1.0, 10), axis=0)
res = []
friends = datum
for dist in try_distances:
    friends = galenv.nn_finder(friends, datum[chosen_ind], dist)
    res.append(len(friends) - 1)

In [ ]:
plt.plot(try_distances, res)
plt.xlabel('distance in angular coordinates')
plt.ylabel('number of neighbors within distance')

### Examine distribution of environment values

In [ ]:
def calc_env(ind):
    res = [subsamples[s]['CATAID'].values[ind]]
    friends = data
    for dist in try_distances:
        friends = galenv.nn_finder(friends, data[ind], dist)
        res.append(len(friends))
    return res

Only execute this cell once, because it's slow!

In [ ]:
# do this for all 4 fields and 19 redshifts separately
all_envs = []
for s in range(len(subsamples)):
    print(lens[s])
    if lens[s] == 1:
        envs_in_field = [[subsamples[s]['CATAID'].values[0]] + [1] * len(try_distances)]
    else:
        data = np.vstack((subsamples[s]['DEC'], [subsamples[s]['RA']])).T
        nps = mp.cpu_count()
        pool = mp.Pool(nps - 1)
        envs_in_field = pool.map(calc_env, range(len(data)))
        pool.close()
    all_envs = all_envs + envs_in_field

In [ ]:
envs_arr = np.array(all_envs)
envs_df = pd.DataFrame(data=envs_arr, index = envs_arr[:, 0], columns = ['CATAID']+[str(i) for i in try_distances])

df = pd.merge(envs_df, zdf, on='CATAID')
df.to_csv('enviros.csv')

Once you've executed the above once, comment it out and execute the following, to skip the slow step.

In [ ]:
zenvdf = pd.read_csv('enviros.csv')

In [ ]:
zenvdf.columns

Now we have the number of neighbors for all galaxies!

In [ ]:
plt.figure(figsize=(10, 8))
orig_distances = np.flip(try_distances, axis=0)
for i in range(len(orig_distances)):
    plt.violinplot(envs_df[str(orig_distances[i])], positions=[i], vert=False)
plt.yticks(range(len(orig_distances)), np.around(orig_distances, 3))
plt.semilogx()
plt.ylabel('distance')
plt.xlabel('number of neighbors')
#plt.ylim(0.99, 10)

It remains to be seen whether the multimodal distributions in some bins are physical or not.
(They look like a problem with smoothing over a discrete variable.)
We'll try plotting them as a function of field and redshift next.

# Next step:

## Constructing the redshift-environment-SED/color relationship

In [ ]:
import glob

In [ ]:
def redshift_df(str_zbin):
    r_files = glob.glob('SpecObjPhot/SpecObjPhot*.csv')
    f = [s for s in r_files if str_zbin in s]
    phodf = pd.read_csv(f[0])
    phodf = phodf.drop(columns=['GAMA_NAME', 'IC_FLAG', 
                            'N_SPEC', 'N_GAMA_SPEC', 'DIST', 
                            'SPECID', 'SURVEY', 'SURVEY_CODE',
                            'RA', 'DEC', 'WMIN', 'WMAX', 'Z', 'NQ',
                            'PROB', 'FILENAME', 'URL', 'URL_IMG'])
    df = pd.merge(phodf, zenvdf, on=['CATAID'])
    return df
    

In [ ]:
df_0042 = redshift_df('0.042')
df = df_0042

In [ ]:
### whichdata = ['Z']
corner.corner(np.array([df['RA'], df['DEC']]).T, labels=['RA', 'DEC'], show_titles=True)

In [ ]:
import matplotlib.colors as colors
import matplotlib.cm as cmx
import matplotlib

jet = plt.cm.Spectral #plt.cm.get_cmap('viridis_r')
cNorm  = colors.Normalize(vmin=z_SLICS[0], vmax=z_SLICS[13])
scalarMap = cmx.ScalarMappable(norm=cNorm, cmap=jet)
color=scalarMap.to_rgba(z_SLICS)

In [ ]:
#z_SLICS
try_distances = np.flip(np.geomspace(0.01, 1.0, 10), axis=0)
orig_distances = np.flip(try_distances, axis=0)

In [ ]:
fig, ax = plt.subplots(figsize=(15,10))
#plt.figure(figsize=(15, 15))
for n, z in enumerate(z_SLICS):
    df = redshift_df(str(z))
    if len(df) > 0:
        for i in range(len(orig_distances)):
            parts = ax.violinplot(df[str(orig_distances[i])], positions=[i], vert=False)
            np.where(df[str(orig_distances[i])] < 1)
            c = color[n]
            for pc in parts['bodies']:
                pc.set_facecolor(c)
                pc.set_edgecolor(c)
                pc.set_alpha(0.5)
            
            parts['cbars'].set_color(c)
            parts['cbars'].set_alpha(0.7)
            
            parts['cmaxes'].set_color(c)
            parts['cmaxes'].set_alpha(0.7)
            
            parts['cmins'].set_color(c)
            parts['cmins'].set_alpha(0.7)
    else:
        print("I have nothing for you at n=%s, z=%s"%(n,z))



plt.yticks(range(len(orig_distances)), np.around(orig_distances, 3))
ax.semilogx()
ax.set_ylabel('radial distance [deg]', size=15)
ax.set_xlabel('number of neighbors', size=15)

cax, _ = matplotlib.colorbar.make_axes(ax, pad=0.01)
cbar = matplotlib.colorbar.ColorbarBase(cax, cmap=jet, norm=cNorm)
cbar.ax.set_ylabel('redshift', size=12)
#plt.ylim(0.99, 10)

#plt.savefig('spectral_violinplot_neighbor_v_distance.jpeg')

In [ ]:
df = redshift_df(str(z_SLICS[0]))
for i in range(len(orig_distances)):
    dist = str(orig_distances[i])

    plt.figure(figsize=(10, 8))
    sc = plt.scatter(df.loc[(df['lsstg'] > 0) & (df['lsstr'] > 0) & (df['lssti'] > 0)]['lsstg']  
                - df.loc[(df['lsstg'] > 0) & (df['lsstr'] > 0) & (df['lssti'] > 0)]['lsstr'],
               df.loc[(df['lsstg'] > 0) & (df['lsstr'] > 0) & (df['lssti'] > 0)]['lsstr']  
                - df.loc[(df['lsstg'] > 0) & (df['lsstr'] > 0) & (df['lssti'] > 0)]['lssti'],
                c=df.loc[(df['lsstg'] > 0) & (df['lsstr'] > 0) & (df['lssti'] > 0)][dist].values,
                cmap=plt.cm.Spectral, #'viridis_r',
                vmin=1.0,
                vmax=max(df['1.0']),
                norm=matplotlib.colors.LogNorm(),
               alpha=0.4)

    plt.colorbar(sc)
    plt.xlabel('g-r', size=15)
    plt.ylabel('r-i', size=15)
    plt.title('dist = %s' %dist)

In [ ]:
df_0042 = redshift_df('0.042')

In [ ]:
for index, row in df_0042.iterrows():
    #print(index)
    #if index < 100:
    dist = [str(i) for i in orig_distances]
    plt.plot(orig_distances, row[dist], 'ok', alpha=0.5)
plt.xlabel('radial distance')
plt.ylabel('# neighbors')

# Time series clustering 

In [ ]:
from tslearn.clustering import TimeSeriesKMeans

In [ ]:
df_0042 = redshift_df('0.080')
df_0042.columns

In [ ]:
X = df_0042.loc[(df_0042['lsstr'] > 0) & (df_0042['lssti'] > 0) & (df_0042['lsstz'] > 0), 
                [#'lsstg', 'lsstg_err', 
                 #'lsstr', 'lsstr_err', 
                 #'lssti', 'lssti_err', 
                 #'lsstz', 'lsstz_err', 
                 #'lssty', 'lssty_err',
                 '1.0', '0.5994842503189409',
                 '0.3593813663804626', '0.21544346900318834', 
                 '0.1291549665014884', '0.0774263682681127', 
                 '0.046415888336127774', '0.027825594022071243',
                 '0.016681005372000592', '0.01']].values

df_0042 = df_0042.loc[(df_0042['lsstr'] > 0) & (df_0042['lssti'] > 0) & (df_0042['lsstz'] > 0)]

In [ ]:
km_orig = TimeSeriesKMeans(n_clusters=3, metric="euclidean", max_iter=5, random_state=0).fit(X)

In [ ]:
km_orig.cluster_centers_.shape

In [ ]:
km_dba = TimeSeriesKMeans(n_clusters=3, metric="dtw", max_iter=5,
                          max_iter_barycenter=5,
                           random_state=0).fit(X)

In [ ]:
km_dba.cluster_centers_.shape

In [ ]:
km_sdtw = TimeSeriesKMeans(n_clusters=3, metric="softdtw", max_iter=5,
                            max_iter_barycenter=5,
                            metric_params={"gamma": .5},
                            random_state=0).fit(X)

In [ ]:
wh_1, = np.where(km_orig.labels_ == 0)
wh_2, = np.where(km_orig.labels_ == 1)
wh_3, = np.where(km_orig.labels_ == 2)

In [ ]:
plt.plot(df_0042.iloc[wh_1]['lsstr']-df_0042.iloc[wh_1]['lssti'],
         df_0042.iloc[wh_1]['lssti']-df_0042.iloc[wh_1]['lsstz'], 
         'o', alpha=0.4, label='1st Cluster')

plt.plot(df_0042.iloc[wh_2]['lsstr']-df_0042.iloc[wh_2]['lssti'],
         df_0042.iloc[wh_2]['lssti']-df_0042.iloc[wh_2]['lsstz'],
         'o', alpha=0.2, label='2nd Cluster')

plt.plot(df_0042.iloc[wh_3]['lsstr']-df_0042.iloc[wh_3]['lssti'],
         df_0042.iloc[wh_3]['lssti']-df_0042.iloc[wh_3]['lsstz'],
         'o', alpha=0.1, label='3rd Cluster')

#plt.xlim(-2, 6)
#plt.ylim(-2, 3)

plt.xlabel('r-i')
plt.ylabel('i-z')
plt.legend()

In [ ]:
## Increase number of clusters 
# TBD: intelligently pick a number of cluxters
km = TimeSeriesKMeans(n_clusters=10, metric="euclidean", max_iter=5, random_state=0).fit(X)

In [ ]:
# dump model to pickle so we can call it later
#redshift dependent
os.makedirs('ts_kmeans', exist_ok=True)
filename = 'ts_kmeans/tskmeans_0.080.pkl'
pickle.dump(km, open(filename, 'wb'))

In [ ]:
df_0042['label'] = km.labels_

In [ ]:
df_0042['r-i'] = df_0042['lsstr']-df_0042['lssti']
df_0042['i-z'] = df_0042['lssti']-df_0042['lsstz']

In [ ]:
# Create only the input data columns for Slack
#new = df_0042[['r-i', 'i-z', 'lsstr', 'label']]
#new.to_csv('colors_lsstr_label_0080.csv')
#new

In [ ]:
plt.figure(figsize=(10,10))
for f in range(0, 10):
    #print(f)
    wh_1, = np.where(km.labels_ == f)
    print(len(wh_1))
    plt.scatter(df_0042.iloc[wh_1]['lsstr']-df_0042.iloc[wh_1]['lssti'],
                df_0042.iloc[wh_1]['lssti']-df_0042.iloc[wh_1]['lsstz'], 
                alpha=0.2, #label='%s Cluster' %f,
                c=[f]*len(wh_1),  #c=df.loc[(df['lsstg'] > 0) & (df['lsstr'] > 0) & (df['lssti'] > 0)][dist].values
                cmap=plt.cm.Spectral, #'viridis_r',
                vmin=0,
                vmax=10)
                #norm=matplotlib.colors.LogNorm())

plt.xlim(-2, 3)
plt.ylim(-2, 2)

plt.ylabel('i-z')
plt.xlabel('r-i')
#plt.legend()

In [ ]:
df = df_0042
fig = corner.corner(np.array([df.loc[(km.labels_ == 0)]['lsstr'], 
                        df.loc[(km.labels_ == 0)]['lssti'],
                        df.loc[(km.labels_ == 0)]['lsstz'],
                        #df.loc[(df['lsstg'] > 0) & (df['lsstr'] > 0) & (df['lssti'] > 0)]['lsstg_err'], 
                        #df.loc[(df['lsstg'] > 0) & (df['lsstr'] > 0) & (df['lssti'] > 0)]['lsstr_err'],
                        #df.loc[(df['lsstg'] > 0) & (df['lsstr'] > 0) & (df['lssti'] > 0)]['lssti_err']
                       ]).T, 
              labels=['r', 'i', 'z'], show_titles=True, color='b', plot_density=False,
              plot_contours=False,
              quantiles=[0.5])

corner.corner(np.array([df.loc[(km.labels_ == 4)]['lsstr'], 
                        df.loc[(km.labels_ == 4)]['lssti'],
                        df.loc[(km.labels_ == 4)]['lsstz'],
                        #df.loc[(df['lsstg'] > 0) & (df['lsstr'] > 0) & (df['lssti'] > 0)]['lsstg_err'], 
                        #df.loc[(df['lsstg'] > 0) & (df['lsstr'] > 0) & (df['lssti'] > 0)]['lsstr_err'],
                        #df.loc[(df['lsstg'] > 0) & (df['lsstr'] > 0) & (df['lssti'] > 0)]['lssti_err']
                       ]).T, 
              labels=['r', 'i', 'z'], show_titles=True, color='r', fig=fig, plot_density=False, 
              plot_contours=False,
              quantiles=[0.5])

corner.corner(np.array([df.loc[(km.labels_ == 9)]['lsstr'], 
                        df.loc[(km.labels_ == 9)]['lssti'],
                        df.loc[(km.labels_ == 9)]['lsstz'],
                        #df.loc[(df['lsstg'] > 0) & (df['lsstr'] > 0) & (df['lssti'] > 0)]['lsstg_err'], 
                        #df.loc[(df['lsstg'] > 0) & (df['lsstr'] > 0) & (df['lssti'] > 0)]['lsstr_err'],
                        #df.loc[(df['lsstg'] > 0) & (df['lsstr'] > 0) & (df['lssti'] > 0)]['lssti_err']
                       ]).T, 
              labels=['r', 'i', 'z'], show_titles=True, color='m', fig=fig, plot_density=False, 
              plot_contours=False,
              quantiles=[0.16, 0.84])


# Extract the axes
#axes = np.array(figure.axes).reshape((ndim, ndim))

# Loop over the diagonal
#for i in range(ndim):
#    ax = axes[i, i]
#    ax.axvline(value1[i], color="g")
#    ax.axvline(value2[i], color="r")
#fig.savefig('tsclustering.pdf')

## Generate 3D Multivariate normal 

Notes:
tavy_resampled is drawing `N_resamples` number of samples from our distribution. If the standard deviation on the mu's and Sigma's are low, then stats.multivariate_normal is fine. If there are large errors, then the tavy_resampled take these errors into account when drawing. 

mu_trans, sigma, L_Omega, L_Sigma, Omega are by-products of the fits. There can be nan's in L_Omega and L_Sigma as they are triangle matrices.

mu[1, 2, 3] = 3 means
Sigma[] = covariance matrix

In [ ]:
import pystan
import pickle

In [ ]:
df = pd.read_csv('colors_lsstr_label_0080.csv')

In [ ]:
df_l = df.loc[df['label']==1]

In [ ]:
N = len(df_l)

vals = df_l[[ 'r-i', 'i-z', 'lsstr']].values

N_re = 1

dat = {
    'N': N,
    'D': 3,
    'N_resamples': N_re,
    'tavy': vals,
}

In [ ]:
## You only need this ONE model to fix all of the data no matter what label or redshift you have!
try:
    sm = pickle.load(open('3D_Gaussian_model.pkl', 'rb'))
except:
    sm = pystan.StanModel(file='some_dimensional_gaussian.stan')

In [ ]:
# Save the model so that you don't have to keep regenerating it. 
#with open('3D_Gaussian_model.pkl', 'wb') as f:
#    pickle.dump(sm, f)

In [ ]:
fit = sm.sampling(data=dat, iter=1000, chains=4, warmup=500) 

In [ ]:
print(fit.stansummary())

In [ ]:
summary = pd.DataFrame(fit.summary(pars=['mu', 'Sigma'])['summary'], 
             columns=fit.summary(pars=['mu', 'Sigma'])['summary_colnames'])

In [ ]:
summary['Names'] = fit.summary(pars=['mu', 'Sigma'])['summary_rownames']

In [ ]:
os.makedirs('fit_summaries', exist_ok=True)
summary.to_csv('fit_summaries/summary_0.080_label_9.csv') ## 
summary

In [ ]:
mus = summary.iloc[:3]['mean'].values

plt.figure()
plt.hist(df_l['r-i'], bins=np.arange(-0.5, 1.5, 0.15), alpha=0.5, label='r-i')
plt.hist(df_l['i-z'], bins=np.arange(-0.5, 1.5, 0.15), alpha=0.5, label='i-z')
#plt.hist(df_l['i-z'])

plt.axvline(mus[0], color='k')
plt.axvline(mus[1], color='k')
plt.legend()

plt.figure()
plt.hist(df_l['lsstr'], label='lsstr')

plt.axvline(mus[2], color='k')
plt.legend()

## Since the means are so well defined, we will use scipy to draw random values

In [ ]:
from scipy.stats import multivariate_normal

In [ ]:
mus = summary.iloc[:3]['mean'].values
cov = summary.iloc[3:]['mean'].values.reshape(3, 3)

rando = multivariate_normal.rvs(mus, cov)

In [ ]:
plt.figure()
plt.hist(df_l['r-i'], bins=np.arange(-0.5, 1.5, 0.15), alpha=0.5, label='r-i')
plt.hist(df_l['i-z'], bins=np.arange(-0.5, 1.5, 0.15), alpha=0.5, label='i-z')
#plt.hist(df_l['i-z'])

plt.axvline(mus[0], color='k', label='fit means')
plt.axvline(mus[1], color='k')

plt.axvline(rando[0], color='grey', lw=5, alpha=0.5, label='Random Draw')
plt.axvline(rando[1], color='grey', lw=5, alpha=0.5)
plt.xlabel('color [mag]')
plt.legend()

plt.figure()
plt.hist(df_l['lsstr'], label='lsstr')

plt.axvline(mus[2], color='k', label='fit means')

plt.axvline(rando[2], color='grey', alpha=0.5, lw=5, label='Random Draw')
plt.xlabel('lsstr [mag]')
plt.legend()